In [32]:
import pandas as pd
import numpy as np
from datetime import datetime

In [33]:
movie_all = pd.read_csv('data/all_movie.csv')
movie_info = pd.read_csv('data/movie_info.tsv',sep = '\t')
review = pd.read_csv('data/reviews.tsv',sep = '\t',encoding = 'unicode_escape')

In [34]:
movie_all_cleaned = movie_all.drop(['Cast 4','Cast 3','Cast 5','Cast 6','Director 2','Director 3','Writer 2','Writer 3','Writer 4'],axis = 1)

In [35]:
movie_info_cleaned = movie_info.drop(['currency'],axis = 1)

In [36]:
merged = pd.merge(movie_all_cleaned,movie_info_cleaned,how = 'right',left_on=['Director 1','Writer 1','Runtime'],right_on= ['director','writer','runtime']).drop_duplicates()
merged = merged.drop(['Rating','Genre','Release Date','dvd_date','Director 1','Writer 1'],axis = 1)
merged = merged.drop_duplicates()

In [37]:
movie_final = merged.drop(['Studio','Runtime','Description','Year'],axis = 1)

In [38]:
movie_final['runtime'] = movie_final['runtime'].apply(lambda x: x if pd.isna(x) else float(x[:-8]))

In [39]:
def convert_rating(col):
    scores = [1,0.95,0.85, 0.80, 0.75,0.65,0.60,0.55,0.45,0.40,0.35,0.25,0.20,0.15]
    grades = ['A+','A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-','F+','F']
    if(pd.isna(col)):
        return col
    x = col.split('/')
    try:
        num = float(x[0])
        de = float(x[1])
        if(num < de):
            return num/de
        else:
            return 1.0
    except ValueError:
        try:
            i = grades.index(x[0])
            return scores[i]
        except ValueError:
            return 0.0
    except IndexError:
        return np.abs(float(x[0]) / 10)

In [40]:
review_final = review.copy()
review_final['rating'] = review['rating'].apply(convert_rating)

In [41]:
review_final['fresh'] = review['fresh'].apply(lambda x:1 if x =='fresh' else 0)
movie_final['theater_date'] = movie_final['theater_date'].apply(lambda x:x if pd.isna(x) else datetime.strptime(x,'%b %d, %Y').strftime('%Y-%m-%d'))

In [42]:
movie_final = movie_final.replace('', np.nan, regex=True)
review_final = review_final.replace('', np.nan, regex=True)

In [43]:
review_final.to_csv('cleaned_review.csv',index=False)

In [13]:
movie_final.to_csv('cleaned_movie.csv',index=True)

In [14]:
movie_all.head(5)

,Cast 1,Cast 2,Cast 3,Cast 4,Cast 5,Cast 6,Description,Director 1,Director 2,Director 3,...,Rating,Release Date,Runtime,Studio,Title,Writer 1,Writer 2,Writer 3,Writer 4,Year
0,Brendan Fraser,John Hannah,Maria Bello,Michelle Yeoh,Jet Li,Russell Wong,The Fast and the Furious director Rob Cohen co...,Rob Cohen,Simon Duggan,Director Not Available,...,PG-13,7/24/2008,112 minutes,Universal Pictures,The Mummy: Tomb of the Dragon Emperor,Alfred Gough,Miles Millar,Writer Not Available,Writer Not Available,2008
1,Brett Granstaff,Diahann Carroll,Lara Jean Chorostecki,Roddy Piper,T.J. McGibbon,James Preston Rogers,The journey of a professional wrestler who bec...,Warren P. Sonoda,Director Not Available,Director Not Available,...,PG-13,1/8/2016,111 minutes,Freestyle Releasing,The Masked Saint,Scott Crowell,Brett Granstaff,Writer Not Available,Writer Not Available,2016
2,Leslie Nielsen,Nicollette Sheridan,Andy Griffith,Marcia Gay Harden,John Ales,Barry Bostwick,"Dead pan Leslie Nielsen stars as Dick Steele, ...",Rick Friedberg,Director Not Available,Director Not Available,...,PG-13,5/24/1996,80 minutes,Hollywood Pictures,Spy Hard,Rick Friedberg,Dick Chudnow,Jason Friedberg,Aaron Seltzer,1996
3,Martina Gedeck,Moritz Bleibtreu,Johanna Wokalek,Bruno Ganz,Nadja Uhl,Jan Josef Liefers,Director Uli Edel teams with screenwriter Bern...,Uli Edel,Director Not Available,Director Not Available,...,R,9/25/2008,149 minutes,Vitagraph Films,Der Baader Meinhof Komplex (The Baader Meinhof...,Uli Edel,Bernd Eichinger,Writer Not Available,Writer Not Available,2008
4,Martin Sheen,Marlon Brando,Robert Duvall,Frederic Forrest,Sam Bottoms,Laurence Fishburne,One of a cluster of late-1970s films about the...,Francis Ford Coppola,Director Not Available,Director Not Available,...,R,1/1/1979,153 minutes,United Artists,Apocalypse Now,John Milius,Francis Ford Coppola,Michael Herr,Writer Not Available,1979


In [17]:
search_engine = movie_all[['Description','Title','Cast 1','Director 1','Writer 1','Year','Genre']]

In [18]:
search_engine.drop_duplicates().to_csv('search.csv',index=True)

In [55]:
avg = review_final.groupby(['id']).mean()['fresh']